# Uppgift 1

In [1]:
import pandas as pd
import seaborn as sns
import hashlib as hl
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append("..")
from data_utils import *
import matplotlib.image as mpimg
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# FutureWarning handling
pd.options.mode.copy_on_write = True

df = read_athlete_events("/home/albot/coding/repos/Projekt-OS-Norge/athlete_events.csv")
df = hash_column(df, "Name").drop(columns=["ID"]).rename(columns={"Name(Hash Value)": "Hash"})
nor = df[df["NOC"] == "NOR"]

In [2]:
def medal_counter(df=nor, col="Games"):
    df_medal = df.copy()
    # df_medal["Medal"] = df_medal["Medal"].fillna("No Medal")          # neither one survives, if the other one lives
    # df_medal = df_medal.dropna(subset=["Medal"])                      # neither one survives, if the other one lives
    df_medal = df_medal.drop_duplicates(subset=["Event", "Games", "Team", "Medal"])
    medal_count = df_medal.groupby([col, "Medal"]).size().unstack(fill_value=0)
    medal_count["Total"] = medal_count[["Bronze", "Silver", "Gold"]].sum(axis=1)
    medal_count = medal_count.reindex(columns=["Bronze", "Silver", "Gold", "Total"])
    return medal_count

In [3]:
nor_medals = medal_counter()
nor_medals.head(11)

Medal,Bronze,Silver,Gold,Total
Games,,,,
1900 Summer,3,2,0,5
1904 Summer,0,0,2,2
1906 Summer,0,1,1,2
1908 Summer,3,3,2,8
1912 Summer,5,1,4,10
1920 Summer,9,10,13,32
1924 Summer,3,2,5,10
1924 Winter,6,7,4,17
1928 Summer,1,2,1,4


## Gender distribution  

### Perhaps  
* age  
* medals  
* px subplot pie chart of women w/ medals vs women participating, same for men  
* sns barplot (countplot?) of women particiapting through the years
* best sports

In [4]:
def norwegian_gender_age_distribution(df=nor):

    df_age = df.copy()
    df_age = df_age.drop_duplicates(subset=["Games", "Hash"])       # seems to show the same plot despite dropping thousands of duplicates, why?
    # df_age.info()                                                 # TODO: shows 3174 entries but online I find 2108 olympic athletes from Norway

    fig = px.histogram(df, x="Age", color="Sex", 
                       barmode="overlay", 
                       title="Ages of Norwegian Olympic Athletes", 
                       labels={"count": "Amount", "Sex": "Gender"},
                       color_discrete_sequence=["forestgreen", "orange"])
    fig.update_traces(marker_line_width=1.5)
    
    fig.show()
    # return fig


norwegian_gender_age_distribution()

In [5]:
def norwegian_medals_gender(df=nor,col="Games"):

    nor_wom = nor[nor["Sex"] == "F"]
    nor_men = nor[nor["Sex"] == "M"]
    nor_medals_wom = medal_counter(nor_wom, col).sort_values(by=col)
    nor_medals_men = medal_counter(nor_men, col).sort_values(by=col)

    fig = px.line(nor_medals.reset_index(), x=col, y="Total", color_discrete_sequence=["crimson"], labels={"Total": "Overall"}, title="Norwegian Olympic medals")
    fig.add_scatter(x=nor_medals_men.reset_index()[col], y=nor_medals_men["Total"], mode="lines", name="Men", line=dict(color="forestgreen"))
    fig.add_scatter(x=nor_medals_wom.reset_index()[col], y=nor_medals_wom["Total"], mode="lines", name="Women", line=dict(color="orange"))
    fig.update_layout(xaxis_title="Year", yaxis_title="Number of medals", legend_title_text="Category")
    fig.update_xaxes(tickangle=-90)
    
    fig.show()
    # return fig


norwegian_medals_gender()

# FIXME: t ex, 1920 har overall 32, men 32 och women 1. 2014 har overall 26, men 14 och women 13. 
# UPDATE: verkar vara löst i versionen som Philip gjorde, inte hittat felet/lösningen dock 

In [6]:
# Defines a variable for later use

def norwegian_participants_gender_df(df=nor, col="Games"):

    nor_wom = nor[nor["Sex"] == "F"]
    nor_men = nor[nor["Sex"] == "M"]
    nor_athletes = nor.groupby(col)["Hash"].nunique().reset_index(name="All")
    nor_athletes_wom = nor_wom.groupby(col)["Hash"].nunique().reset_index(name="Women")
    nor_athletes_men = nor_men.groupby(col)["Hash"].nunique().reset_index(name="Men")
    nor_athletes = nor_athletes.merge(nor_athletes_wom, on=col, how="left").fillna(0)
    nor_athletes = nor_athletes.merge(nor_athletes_men, on=col, how="left").fillna(0)
    nor_athletes[["Women", "Men"]] = nor_athletes[["Women", "Men"]].astype(int)

    return nor_athletes


nor_athletes = norwegian_participants_gender_df()

In [7]:
def norwegian_participants_gender(df=nor_athletes, col="Games"):
    fig = px.bar(df, x=col, y=["Women", "Men"], 
                color_discrete_sequence=["orange", "forestgreen"], 
                title="Norwegian athletes in the Olympics", 
                labels={"value": "Participants", "variable": "Gender", "Games": ""})
    fig.update_xaxes(tickangle=-90)

    fig.show()
    # return fig


norwegian_participants_gender()

In [8]:
def norwegian_gender_percentage(df=nor_athletes, col="Games"):

    df["Women%"] = ((df["Women"] / df["All"]) * 100).round(0).astype(int)
    df["Men%"] = ((df["Men"] / df["All"]) * 100).round(0).astype(int)
    df = df.reindex(columns=["Games", "All", "Women", "Women%", "Men", "Men%"])
    fig = px.line(df, x="Games", y=["Women%", "Men%"], color_discrete_sequence=["orange", "forestgreen"], labels={"value": "Percentage", "variable": "Gender", "Games": " "}, title="Women vs men in the Norwegian Olympics teams")
    fig.update_xaxes(tickangle=-90)
    
    fig.show()
    # return fig


norwegian_gender_percentage()

In [9]:
def norwegian_medals_sport_per_games(df=nor, col="Games"):

    nor_medals_sport = df.copy()
    nor_medals_sport = nor_medals_sport.drop_duplicates(subset=["Event", "Games", "Team", "Medal"])
    nor_medals_sport = nor_medals_sport.groupby([col, "Sport"])["Medal"].count().unstack(fill_value=0)
    nor_medals_sport = nor_medals_sport.reset_index()

    fig = px.bar(nor_medals_sport, x=col, y=nor_medals_sport.columns[1:], 
                title="Norwegian Olympic medals by sport", 
                labels={"Total": "Medals", "index": "Sport", "Games": "", "value": "Medals"}, 
                color_discrete_sequence=px.colors.qualitative.Plotly)                               # TODO: individual colours per sport or unique for the most prominent ones
    fig.update_xaxes(tickangle=-90)
    
    fig.show()
    # return fig


norwegian_medals_sport_per_games()

In [31]:
def norwegian_medals_by_sport(df=nor, headline="Norwegian Olympic medals by sport"):

    def sports_medals():

        sport_medal = df.copy()
        sport_medal = sport_medal.drop_duplicates(subset=["Event", "Games", "Team", "Medal"])
        sport_medal = sport_medal.groupby(["Sport", "Medal"]).size().unstack(fill_value=0)
        sport_medal["Total"] = sport_medal.sum(axis=1)
        sport_medal = sport_medal.reindex(columns=["Bronze", "Silver", "Gold", "Total"])
        sport_medal = sport_medal.sort_values(by="Total", ascending=False)
        sport_medal = sport_medal.reset_index()
        
        return sport_medal
    

    nor_sports_medals = sports_medals()

    fig = px.bar(nor_sports_medals.head(10), x="Sport", y="Total", title=headline, labels={"Total": "Medals", "Sport": ""}, color="Sport")
    fig.update_xaxes(tickangle=-90)
    
    fig.show()
    # return fig


nor_wom = nor[nor["Sex"] == "F"]
nor_men = nor[nor["Sex"] == "M"]

norwegian_medals_by_sport()
norwegian_medals_by_sport(nor_wom, "Norwegian Olympic medals by sport for women")
norwegian_medals_by_sport(nor_men, "Norwegian Olympic medals by sport for men")

In [13]:
def norwegian_medals_decade(df=nor):

	nor_wom = nor[nor["Sex"] == "F"]
	nor_men = nor[nor["Sex"] == "M"]
	nor_medals = medal_counter()
	nor_medals_wom = medal_counter(nor_wom, "Games").sort_values(by="Games")
	nor_medals_men = medal_counter(nor_men, "Games").sort_values(by="Games")

	nor_medals_decade = nor_medals.reset_index()
	temp_men = nor_medals_men.reset_index()
	temp_wom = nor_medals_wom.reset_index()
	nor_medals_decade = nor_medals_decade[["Games", "Total"]]
	temp_men = temp_men[["Games", "Total"]]
	temp_wom = temp_wom[["Games", "Total"]]

	nor_medals_decade = nor_medals_decade.merge(temp_men, on="Games", how="left")
	nor_medals_decade = nor_medals_decade.merge(temp_wom, on="Games", how="left").fillna(0)
	nor_medals_decade["Total"] = nor_medals_decade["Total"].astype(int)
	nor_medals_decade = nor_medals_decade.rename(columns={"Total_x": "Medals", "Total_y": "Men", "Total": "Women"})
	nor_medals_decade["Decade"] = nor_medals_decade["Games"].apply(lambda row: int(row[:3] + "0"))
	nor_medals_decade = nor_medals_decade.groupby("Decade", as_index=False)[["Medals", "Men", "Women"]].sum()

	# the below is the result of a Copilot prompt: "Using plotly express and pandas, how can I plot multiple pie plots with subplots from row values of a dataframe?"
	fig = make_subplots(rows=1, cols=len(nor_medals_decade), specs=[[{"type": "domain"}] * len(nor_medals_decade)],
						subplot_titles=[f"{decade}s" for decade in nor_medals_decade["Decade"]])

	for i, row in nor_medals_decade.iterrows():
		fig.add_trace(go.Pie(labels=["Men", "Women"], values=[row["Men"], row["Women"]], name=f"{row["Decade"]}s",
							marker_colors=["forestgreen", "orange"]), 1, i+1)
	# the above is the result of a Copilot prompt: "Using plotly express and pandas, how can I plot multiple pie plots with subplots from row values of a dataframe?"

	fig.update_layout(title_text="Medals won by male and female athletes per decade")
	
	fig.show()
	# return fig


norwegian_medals_decade()

In [14]:
def medal_coloured_bars(df=nor, col="Games"):
    
    def medal_counter():
        df_medals = df.copy()
        df_medals = df_medals.drop_duplicates(subset=["Event", "Games", "Team", "Medal"])
        df_count = df_medals.groupby([col, "Medal"]).size().unstack(fill_value=0)
        df_count["Total"] = df_count[["Bronze", "Silver", "Gold"]].sum(axis=1)
        df_count = df_count.reindex(columns=["Bronze", "Silver", "Gold", "Total"])
        return df_count
    

    df_medal_count = medal_counter()
    df_medal_count = df_medal_count.reset_index()

    fig = px.bar(df_medal_count, 
             x=col, y=["Bronze", "Silver", "Gold"], 
             title="Norwegian Olympic medals", 
             labels={"Total": "Medals", "index": "Sport", "Games": "", "value": "Medals", "variable": ""}, 
             color_discrete_sequence=["#cd7f32", "#c0c0c0", "#ffd700"])
    fig.update_xaxes(tickangle=-90)

    fig.show()
    # return fig


medal_coloured_bars()

## Norway and the rest

In [ ]:
def norwegian_medals_season(df=nor):
    
    nor_medals_winter = df[df["Season"] == "Winter"].dropna(subset=["Medal"]).drop_duplicates(subset=["Event", "Games", "Team", "Medal"])
    nor_medals_summer = df[df["Season"] == "Summer"].dropna(subset=["Medal"]).drop_duplicates(subset=["Event", "Games", "Team", "Medal"])
    medals_winter = nor_medals_winter.groupby("Games")["Medal"].count().reset_index(name="Medals")
    medals_summer = nor_medals_summer.groupby("Games")["Medal"].count().reset_index(name="Medals")

    fig = make_subplots(rows=1, cols=2, subplot_titles=("Winter games", "Summer games"))
    fig.add_trace(go.Bar(x=medals_winter["Games"], y=medals_winter["Medals"], marker_color="skyblue"), row=1, col=1)
    fig.add_trace(go.Bar(x=medals_summer["Games"], y=medals_summer["Medals"], marker_color="orange"), row=1, col=2)
    fig.update_layout(title_text="Norwegian seasonal medals", showlegend=False)         # TODO: label name for y axis
    fig.update_xaxes(tickangle=-90)
    
    fig.show()
    # return fig


norwegian_medals_season()

In [39]:
def top_medals_winter():
    winter_medals = df[df["Season"] == "Winter"].dropna(subset=["Medal"])
    winter_medals = winter_medals.drop_duplicates(subset=["Event", "Games", "Team", "Medal"])
    winter_medals = winter_medals.groupby("NOC")["Medal"].count().reset_index(name="Medals")
    winter_medals = winter_medals.sort_values(by="Medals", ascending=False)

    fig = px.bar(winter_medals.head(10), x="NOC", y="Medals", title="Olympic winter game medals by country", labels={"NOC": "", "Medals": "Amount"}, color="NOC")
    fig.update_layout(showlegend=False)
    
    fig.show()
    # return fig


top_medals_winter()

In [57]:
# a graph showing the winter medals for norway vs Soviet (NOC="URS") + Russia (NOC="RUS")
def nor_vs_rus_winter_medals():

    winter_medals = df[df["Season"] == "Winter"].dropna(subset=["Medal"])
    winter_medals = winter_medals.drop_duplicates(subset=["Event", "Games", "Team", "Medal"])
    winter_medals = winter_medals.groupby("NOC")["Medal"].count().reset_index(name="Medals")
    winter_medals = winter_medals.sort_values(by="Medals", ascending=False)
    versus = winter_medals[winter_medals["NOC"].isin(["NOR", "RUS", "URS"])]
    versus.iloc[2, 1] = versus.iloc[2, 1] + versus.iloc[1, 1]
    versus = versus.drop(versus.index[1])

    fig = px.bar(versus, x="NOC", y="Medals", title="Norway vs combined medals for Russia and Soviet Union", labels={"NOC": "", "Medals": "Amount"}, color="NOC")

    fig.show()
    # return fig


nor_vs_rus_winter_medals()

In [ ]:
# scatter plot with age of medal winners